In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib 
from matplotlib import pyplot as plt
from scipy.special import erfcinv

In [2]:
data_names = np.loadtxt("../Training/simulation/invert/invert.ls", delimiter=",",dtype=str)

In [ ]:
for name in data_names[:15000]:
    
    try:
        first_bls = np.loadtxt("../Training/simulation/invert/" + name[:-3]+"blsanal")[0, 1:]
        first_lc = np.loadtxt("../Training/simulation/invert/"+name[:-4]+".txt")
    except:
        print "Faulty file"
        continue 
   
    phase = ((first_lc[:,0] -first_bls[1])/first_bls[0])%1
    depth = first_bls[5]
    transit_duration = first_bls[6]
    ingress_duration = first_bls[7]*first_bls[6]*1.5


    #find white noise, and remove primary signal
    white_noise = np.std(first_lc[:,1][(phase>transit_duration/2) & (phase<1-transit_duration/2)])
    mean_out_transit_flux = np.mean(first_lc[:,1][(phase>transit_duration/2) & (phase<1-transit_duration/2)])
    #remove in transit portion

    cleaned_vals = np.empty(shape=[0,3])
    slope = depth /ingress_duration

    
    for i, vals in enumerate(first_lc):
        vals =vals.reshape(1,3)
        if phase[i] <transit_duration-ingress_duration or phase[i]>1-transit_duration+ingress_duration:
             vals[0][1] = vals[0][1]-depth
        elif ((phase[i] > transit_duration-ingress_duration  and phase[i]<transit_duration)) : 
             vals[0][1] -= (ingress_duration - phase[i]) *slope

        elif ((phase[i]>1-transit_duration) and (phase[i]<1-transit_duration+ingress_duration)):
             vals[0][1] -= (ingress_duration - (1 - phase[i])) *slope

        cleaned_vals = np.append(cleaned_vals, vals, axis = 0)



    #using transit width, search through phase space to find secondary and tertiary signals
    comb = zip(phase, cleaned_vals[:,1]) 
    comb.sort()
    phase_time, total_flux = zip(*comb)
    total_flux = np.asarray(total_flux).reshape((len(total_flux),1))

    mod_sig_sec_dv = 0
    mod_sig_pos_dv = 0
    sec_loc = 1
    phase_time = np.asarray(phase_time)

    cut_phase =  phase_time#[(phase_time > transit_duration) & (phase_time<(1-transit_duration))]
    cut_flux = total_flux#[(phase_time > transit_duration) & (phase_time<(1-transit_duration))]

    for i, vals in enumerate(cut_phase):
        section = cut_flux[(cut_phase>cut_phase[i]) & (cut_phase<cut_phase[i]+transit_duration*2)]
        depth_two = np.mean(section) - mean_out_transit_flux
        if depth_two == max( mod_sig_sec_dv, depth_two):
            sec_loc = phase_time[i]
        mod_sig_sec_dv = max(mod_sig_sec_dv, depth_two)
        mod_sig_pos_dv = min(mod_sig_pos_dv, depth_two) 




    for i, vals in enumerate(cut_phase):
        if (vals< sec_loc+transit_duration) and (vals>sec_loc-transit_duration):
            cut_flux[i] -= mod_sig_sec_dv

    mod_sig_ter_dv = 0
    for i, vals in enumerate(cut_phase):
        section = cut_flux[(cut_phase>cut_phase[i]) & (cut_phase<cut_phase[i]+transit_duration*2)]
        sig = np.mean(section) - mean_out_transit_flux
        mod_sig_ter_dv = max(mod_sig_sec_dv, sig)
                                        
    mod_sig_sec_dv = mod_sig_sec_dv - mean_out_transit_flux


    #red noise
    #red noise^2 = white noise /npoiint + red noise
    #inverse relationship
    #try with three timescales, one, two, and threee t durations
    #look at portions outside of transit?
    red_noise_1 = []
    red_noise_2 = []
    red_noise_3 = []

    red_noise_1_p = []
    red_noise_2_p = []
    red_noise_3_p = []
    print len
    for i, vals in enumerate(cut_phase[:-1]):
        section1 = cut_flux[(cut_phase>cut_phase[i]) & (cut_phase<cut_phase[i]+transit_duration)]
        section2 = cut_flux[(cut_phase>cut_phase[i]) & (cut_phase<cut_phase[i]+transit_duration*2)]
        section3 = cut_flux[(cut_phase>cut_phase[i]) & (cut_phase<cut_phase[i]+transit_duration*3)]
        red_noise_1_p.append(section1.shape[0])
        red_noise_2_p.append(section2.shape[0])
        red_noise_3_p.append(section3.shape[0])
        red_noise_1.append(np.var(section1))
        red_noise_2.append(np.var(section2))
        red_noise_3.append(np.var(section3))
    
    N_p_1 = np.median(red_noise_1_p)
    N_p_2 = np.median(red_noise_2_p)
    N_p_3 = np.median(red_noise_3_p)

    white_noise_2 = np.abs((np.mean(red_noise_1) - np.mean(red_noise_2))* (N_p_1 * N_p_2) / (N_p_1+N_p_2))

    print "Red noise:", np.mean(red_noise_1),  " White noise", white_noise_2
    red_noise = np.sqrt(np.mean(red_noise_1) - white_noise_2/N_p_1)

    #from scipy.special import erfcinv

    period = first_bls[0]
    n_trans = first_bls[14]


    mod_sig_pri_dv = depth
    mod_fred_dv = red_noise / white_noise # ratio between the two
    print red_noise, white_noise
    mod_fa1_dv = np.sqrt(2) * erfcinv(transit_duration/(period* n_trans))
    mod_fa2_dv = np.sqrt(2) * erfcinv(transit_duration/(period))

    modshiftval1_dv = mod_sig_pri_dv/mod_fred_dv - mod_fa1_dv
    modshiftval2_dv = mod_sig_pri_dv - mod_sig_ter_dv-mod_fa2_dv
    modshiftval3_dv = mod_sig_pri_dv - mod_sig_pos_dv - mod_fa2_dv
    print mod_sig_sec_dv, mod_fred_dv, mod_fa1_dv
    modshiftval4_dv = mod_sig_sec_dv / mod_fred_dv - mod_fa1_dv
    modshiftval5_dv = mod_sig_sec_dv - mod_sig_ter_dv - mod_fa2_dv
    modshiftval6_dv = mod_sig_sec_dv-mod_sig_pos_dv - mod_fa2_dv


    vals = [mod_sig_pri_dv, mod_sig_sec_dv, mod_sig_ter_dv, mod_sig_pos_dv, mod_fa1_dv, mod_fa2_dv, mod_fred_dv, modshiftval1_dv, modshiftval2_dv, modshiftval3_dv, modshiftval4_dv, modshiftval5_dv, modshiftval6_dv, white_noise, red_noise]

    total_Vals = np.concatenate([vals, first_bls])

    head_txt = """mod_sig_pri_dv mod_sig_sec_dv mod_sig_ter_dv mod_sig_pos_dv mod_fa1_dv mod_fa2_dv mod_fred_dv modshiftval1_dv modshiftval2_dv modshiftval3_dv modshiftval4_dv modshiftval5_dv modshiftval6_dv white_noise red_noise BLS_Period_1_0 BLS_Tc_1_0 BLS_SN_1_0 BLS_SR_1_0 BLS_SDE_1_0 BLS_Depth_1_0 BLS_Qtran_1_0 BLS_Qingress_1_0 BLS_OOTmag_1_0 BLS_i1_1_0 BLS_i2_1_0 BLS_deltaChi2_1_0 BLS_fraconenight_1_0 BLS_Npointsintransit_1_0 BLS_Ntransits_1_0 BLS_Npointsbeforetransit_1_0 BLS_Npointsaftertransit_1_0 BLS_Rednoise_1_0 BLS_Whitenoise_1_0 BLS_SignaltoPinknoise_1_0 """
    np.savetxt("InvModShiftVals/" + name[:-8] +"_feats_fixed.txt", total_Vals, header=head_txt)
    #break
    print ("\n")

<built-in function len>
Red noise: 1.21105069194e-07  White noise 1.68777141779e-08
0.000347654942424 0.00031544756436
-11.9998024033 1.10210057614 3.09157895886


<built-in function len>
Red noise: 1.20590562961e-07  White noise 1.53148896639e-07
0.000345561129939 0.000309873186247
-11.9997600618 1.11516951216 2.89645540294


<built-in function len>
Red noise: 1.37626175172e-07  White noise 1.85778543765e-08
0.000370532627136 0.000325022910987
-11.9997784537 1.14002002508 3.23682874067


<built-in function len>
Red noise: 1.21495926532e-07  White noise 7.03337281589e-08
0.000347684237342 0.000335626193356
-11.9998784569 1.03592700518 2.9569679835


<built-in function len>
Red noise: 9.94429793845e-08  White noise 7.72333430146e-08
0.00031109445617 0.000330930438008
-11.9998301073 0.940059965602 3.32762050387


<built-in function len>
Red noise: 1.17709575962e-07  White noise 1.09962930656e-07
0.000340575109441 0.000333648057898
-11.9997600153 1.02076155212 3.18957602684


<built-in fu